# [작성중...]Bidirectional LSTM-CRF Models for Sequences Tagging


## 1. Introduction


### 1) Sequence tagging(=Sequence labeling)

&nbsp;길이가 $n$인 sequence, $x=[x_{1}, x_{2},...,x_{n}]$에 대해서, 같은 길이의 $y=[y_{1}, y_{2},...,y_{n}]$를 출력해야 하는 Task를 Sequence tagging(=Sequence labeling)이라고 합니다. 

1. NLP Sequence labeling task
    - speech tagging (POS)
    - chunking
    - named entity recognition (NER)
    ![POS](./res/PR001/pos.jpg)
    
    
2. 다른 분야의 Sequence labeling task
    - DNA 염기서열 분석
    - 동작 인식, 이미지 레이블링, 물체 인식
    - 음석 인식, POS 태깅, 개체 인식, 정보 추출 
    - Sequence로 해석될 수 있는 다양한 종류의 데이터 처리

### 2) Exsisting sequence tagging models
1. Linear statistical models 
    1. Hidden-Markov Models 
    2. Maximum entropy Markov models
    3. Conditional Random Fields (CRF)
    
2. Neural network based models
    1. Convolutional-CRF
    2. Convolutional-RNN,LSTM 
    3. Bidirectoinal-RNN,LSTM

### 3) Proposed models in paper
&nbsp;논문에서 제시한 최종 모델은 BiLSTM-CRM 모델인데, Word Embedding에 덜 의존적이면서, POS, Chunking, NER에서 State of the art 성능을 내었다고 합니다. 

1. LSTM
2. Bi-LSTM
3. LSTM-CRM
4. bi-LSTM-CRM 


## 2. Models
&nbsp;논문을 읽으면서 가장 좋았던 점은 RNN, LSTM 모델들의 정의를 한번더 정리할 수 있었던 점입니다. 지금부터는 RNN, LSTM, CRF 모델들에대한 직관과 수학적 정의를 짚어볼 것입니다. 

### 2.0 RNN Networks
&nbsp;RNN 모델은 언어모델, 음성인식 분야에서 보장된 결과를 많이 만들어 왔습니다. RNN의 특징은 history information을 저장하는 Memory 학습하고, 이용해서 현재의 알맞는 예측, 분류 Output을 출력합니다. 

&nbsp;논문에서 제시된 문제는 named entity recognition (NER)니다. 아래 문장이 입력 시퀀스로 주어졌을때, 각각에 맞는 Named entity를 출력하는 문제입니다. 

입력문장은 "EU rejects German call to boycott British lamb." 이고<br>
Named entity tags는 O(Other) 혹은 Person(PER), Location(LOC), Organization(ORG) 그리고 Miscellaneus (MISC)입니다. 추가로, tag앞에 B-, I-가 붙는데 개체의 위치가 중간인지 처음인지를 나타내는 지표라고 합니다. 

![Figure1 A simple RNN model.](./res/PR001/figure1_RNN.png)

그림과 같이 RNN은 입력층 x, 은닉층 h, 출력층 y로 구성되어 있습니다. 입력부터 출력까지의 연산의 정의는 아래와 같습니다. 

$h(t) = f(U*x(t)+W*h(t-1))$<br>
$y(h) = g(V*h(t))$<br>

여기서 $U$, $W$, $V$는 Connection weights이며, $f()$는 sigmoid, $g()$는 softmat 함수 입니다.

### 2.1 LSTM Networks 
&nbsp;LSTM(Long Short-Term Memory)은 RNN과 동일하지만, 장기적인 종속성을 학습할 수 있는 특징이 있습니다. 
<center><img src="./res/PR001/lstm.png" width="300" height="300"></center>

&nbsp;맨 위에 컨베이너 벨트처럼 흐르는 $C$값이 cell state이며, LSTM은 이 cell state를 보호하고 컨트롤 하기 위한 세 가지 게이트: forget, input, output gate를 통해 vanishing gradient를 방지하고 그래디언트가 효과적으로 흐를 수 있게 합니다.

- __forget gate $f_{t}$__ 는 말그대로 ‘과거 정보를 잊기’위한 게이트다. 시그모이드 함수의 출력 범위는 0 ~ 1 이기 때문에 그 값이 0이라면 이전 상태의 정보는 잊고, 1이라면 이전 상태의 정보를 온전히 기억하게 됩니다.

- __input gate $i_{t}$__는 ‘현재 정보를 기억하기’위한 게이트다. 이 값은 시그모이드 이므로 0 ~ 1 이지만 hadamard product를 하는 $\tilde{c}_{t}$는 hyperbolic tangent 결과이므로 -1 ~ 1 이 된다. 따라서 결과는 음수가 될 \tilde{a} 수도 있습니다.
- __output gate__ $o_{t}$는 최종 결과 $h_{t}$를 위한 게이트이며, cell state의 hyperbolic tangent를 hadamard product한 값이 LSTM의 최종 결과가 된다.

__LSTM 수식__<br>
$f_{t}=  \sigma_{g}(W_{f}x_{t}+U_{f}h_{t-1}+b_{f})$<br>
$i_{t}=  \sigma_{g}(W_{i}x_{t}+U_{i}h_{t-1}+b_{i})$<br>
$i_{o}=  \sigma_{g}(W_{o}x_{t}+U_{o}h_{t-1}+b_{o})$<br>
$c_{t}=f_{t} \circ c_{t-1}+i_{t} \circ \sigma_{c}(W_{c}x_{t}+U_ch_{t-1}+b_{c})$<br>
$h_{t}=o_{t} \circ \sigma_{h}(C_{t})$<br>



### 2.2 Bidirectional LSTM Networks 
&nbsp;__LSTM cel__l이 __단방향 입력 단어들의 문맥__을 학습한다면, __Bidirectional LSTM cell__(이하 BiLSTM)은 특정 시간프레임 안에서 __양방향 입력 단어들의 문맥__을 학습합니다. BiLSTM의 구조는 하나의 LSTM과 Reverse된 LSTM이 병렬로 입출력에 연결되어 있는 형태입니다. 논문에서는 이를 각각 Forward pass 와 Backward pass로 정의합니다.

<center><img src="./res/PR001/BiLSTM.png" width="300" height="300"></center>

&nbsp;Sequence Tagging을 위해 논문에서 BiLSTM을 학습시키는 노하우로 문장의 시작점에서만 BiLSTM의 hidden state를 0으로 초기화 하는 것입니다.



### 2.3 CRF networks 
&nbsp;이웃한 태그(이하 출력 단어) 정보를 학습하는 두가지 방법을 소개합니다.
1. Maximum entrtopy classifier, Maximum entropy Markov models(MEMMs)
2. Conditional Random Fileds(CRF)

&nbsp;위 두가지 선형 통계 모델은 오래전부터 Sequence tagging 분야에서 사랑받아오던 모델입니다. Sequence tagging문제를 $P(y_{1:n}|x_{1:n})$으로 정의한다면, MEMMs와 CRF모델의 수식은 아래와 같습니다. 

__MEMMs__<br>
$$P(y_{1:n}|x_{1:n}) = \prod_{i=1}^n{ \frac{exp( \sum_{j=1}^m f_{j}(x,i,y_{i}, y_{i-1}) )}{\sum_{y'} exp(\sum_{j=1}^m  \lambda _{j}f_{j}(x,i,y'_{i}, y'_{i-1}))}  }$$

__CRF__<br> 
$$P(y_{1:n}|x_{1:n}) = \prod_{i=1}^n{ \frac{exp(\sum_{j=1}^{m}\sum_{i=1}^{n}\lambda _{j}f_{j}(x,i,y_{i}, y_{i-1}))  )}{\sum_{y'}exp(\sum_{j=1}^{m}\sum_{i=1}^{n}\lambda _{j}f_{j}(x,i,y'_{i}, y'_{i-1})))}  }$$

### 2.4 LSTM-CRF networks
&nbsp; 결과적으로 LSTM과 CRF모델을 함께 사용하면, __LSTM Layer는 입력 문장의 단방향 문맥을 학습__하고, __CRF Layer는 출력 Tag 시퀀스의 문맥을 학습__하는 것이 됩니다. __CRF Layer는 state transition matrix__를 파라미터로 가지고 있습니다. 그로인해 현재 Tag를 예측하기 위해 미래의 태그와 과거의 Tag의 관계를 연산하게 됩니다. 아래는 LSTM-CRF network의 수학적 정의입니다.

__LSTM-CRF networks__<br>
$$s([x]_1^T, [i]_1^T, \tilde{\theta}) =   \sum_{t=1}^{T}([A]_{[i]_{t-1},[i]_{t}}+[f_{\theta}]_{[i]_{t-1},[i]_{t}})$$

- LSTM network's input sequence: $[x]_{T}^{1}$ 
- LSTM network's output matrix:  $f_{ \theta }( [x]_{T}^{1})$ 
- Elements of output matrix:  $[f_{\theta}]_{i,t}$
- LSTM network's parameter: $\theta$
- LSTM-CRF networks' parameter :  $\tilde{\theta} =\theta \cup \{[A]_{i,j} \forall i, j\}$
- LSTM-CRF network's output: $[i]_1^T$
- Transoition score: $[A]_{i,j}$<br>
i~j번째까지의 time steps의 연이은 변화를 모델링 하기위한 Matrix score입니다.

아래의 그림은 양방향 LSTM + CRF 모델을 보여줍니다.
<center><img src="./res/PR001/BiSLTM-CRF.png" width="300" height="300"></center>

기존에 CRF 층이 존재하지 않았던 LSTM 모델은 활성화 함수를 지난 시점에서 개체명을 결정했지만, CRF 층을 추가한 모델에서는 활성화 함수의 결과들이 CRF 층의 입력으로 전달됩니다. CRF 층은 레이블 시퀀스에 대해서 Transoition score와 LSTM 모델의 출력을 합쳐 가장 높은 점수를 가지는 시퀀스를 예측합니다.

### 2.5 BI-LSTM-CRF networks 
&nbsp;이론은 LSTM-CRF와 동일하며, 입력문장의 양방향 문맥을 학습한다는 특징이 있습니다. 이 이후부터는 BiLSTM-CRF 모델을 설계하고, 간단한 개체명 Named Entity Recognition을 진행해 보도록 하겠습니다. 

## 4. Experiments 

### 4.1 Data
__논문에서 사용한 데이터__
- [Penn TreeBank(PTB)](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) POS tagging 
- [CoNLL-2000 Shared Task Chunking](https://www.aclweb.org/anthology/W00-0726/)
- [CoNLL-2003 Language-Independent Named Entity Recognition (II)](https://www.clips.uantwerpen.be/conll2003/ner/)

__노트북에서 실험을 위해 사용할 데이터__ 
- [Kaggle dataset: Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus)



### 5 Discussions 

### 6 Conclusions 

### Reference
[1] [Bidirectional LSTM-CRF Models for Sequence Tagging](https://arxiv.org/abs/1508.01991)<br>
[2] [김영민 Sequence Labeling을
위한 기계학습.pdf](http://www.hansung.ac.kr/web/sdkim/kcc2014-nlp-workshop?p_p_id=EXT_BBS&p_p_lifecycle=1&p_p_state=exclusive&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_EXT_BBS_struts_action=%2Fext%2Fbbs%2Fget_file&_EXT_BBS_extFileId=635976)<br>
[3] [Conditional Random Field (CRF) 기반 품사 판별기의 원리와 HMM 기반 품사 판별기와의 차이점](https://lovit.github.io/nlp/2018/09/13/crf_based_tagger/)<br>
[4] [LSTM의 원리와 수식 계산](http://docs.likejazz.com/lstm/)<br>
[5] [양방향 LSTM과 CRF(Bidirectional LSTM + CRF)](https://wikidocs.net/34156)